In [1]:
import gdal, osr
from skimage.graph import route_through_array
import numpy as np


In [ ]:
def raster2array(rasterfn):
    raster = gdal.Open(rasterfn)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    return array  

def coord2pixelOffset(rasterfn,x,y):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3] 
    pixelWidth = geotransform[1] 
    pixelHeight = geotransform[5]
    xOffset = int((x - originX)/pixelWidth)
    yOffset = int((y - originY)/pixelHeight)
    return xOffset,yOffset


In [ ]:
def createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord):   

    # coordinates to array index
    startCoordX = startCoord[0]
    startCoordY = startCoord[1]
    startIndexX,startIndexY = coord2pixelOffset(CostSurfacefn,startCoordX,startCoordY)

    stopCoordX = stopCoord[0]
    stopCoordY = stopCoord[1]
    stopIndexX,stopIndexY = coord2pixelOffset(CostSurfacefn,stopCoordX,stopCoordY)

    # create path
    indices, weight = route_through_array(costSurfaceArray, (startIndexY,startIndexX), (stopIndexY,stopIndexX),geometric=True,fully_connected=True)
    indices = np.array(indices).T
    path = np.zeros_like(costSurfaceArray)
    path[indices[0], indices[1]] = 1
    return path

In [ ]:
def array2raster(newRasterfn,rasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3] 
    pixelWidth = geotransform[1] 
    pixelHeight = geotransform[5]
    cols = array.shape[1]
    rows = array.shape[0]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache() 

In [ ]:
def main(CostSurfacefn,outputPathfn,startCoord,stopCoord):

    costSurfaceArray = raster2array(CostSurfacefn) # creates array from cost surface raster

    pathArray = createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord) # creates path array

    array2raster(outputPathfn,CostSurfacefn,pathArray) # converts path array to raster


if __name__ == "__main__":
    CostSurfacefn = 'CostSurface.tif'
    startCoord = (345387.871,1267855.277)
    stopCoord = (345479.425,1267799.626)
    outputPathfn = 'Path.tif'
    main(CostSurfacefn,outputPathfn,startCoord,stopCoord)

In [ ]:
costSurfaceArray = raster2array(CostSurfacefn)

In [ ]:
costSurfaceArray.shape

In [ ]:
startCoord=(1,3)
stopCoord=(100,5)
pathArray=createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord)

In [ ]:
costSurfaceArray=np.array([[5,3,1],[6,2,7],[9,1,5]]) 
costSurfaceArray

costSurfaceArray=np.array([[5,3,1],[6,2,7],[9,1,5])
costSurfaceArray

In [ ]:
costSurfaceArray=np.random.rand(548,421)
startIndexX=0
startIndexY=0
stopIndexX=costSurfaceArray.shape[1]-1
stopIndexY=costSurfaceArray.shape[0]-1
indices, weight = route_through_array(costSurfaceArray, (startIndexY,startIndexX), (stopIndexY,stopIndexX),geometric=True,fully_connected=True)


In [ ]:
weight

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(costSurfaceArray, cmap='hot', interpolation='nearest')
#plt.plot([[0,0],[1,1]])
for item in indices:
    plt.plot(item[1],item[0], 'bo')
plt.show()

In [ ]:
list(zip(indices,indices[:-1]))

In [ ]:
[[x[0], x[1]] for x in list(zip(indices,indices[1:len(indices)]))]
#indices

In [29]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
citydata = pd.read_csv('CityData.csv')


Exception ignored in: <bound method Env.__del__ of <TimeLimit<ZeppelinEnv<zeppelin-v2>>>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 203, in __del__
    self.close()
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 166, in close
    self._close()
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 290, in _close
    return self.env.close()
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 164, in close
    self.render(close=True)
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 150, in render
    return self._render(mode=mode, close=close)
TypeError: _render() got an unexpected keyword argument 'mode'
Exception ignored in: <bound method Env.__del__ of <__main__.ZeppelinEnv object at 0x7fc75e345dd8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/gym/core.py", line 203, in __del__
    self.close()
  File "/usr/local/lib/python3.5/di

In [ ]:
h5f = h5py.File('METdata.h5', 'r')
train = h5f['train'][:]
test = h5f['test'][:]
h5f.close()

In [ ]:
import numpy
x=numpy.loadtxt(open("test.out", "rb"), delimiter=",")
x.shape

In [ ]:
import numpy
x=numpy.loadtxt(open("test.out", "rb"), delimiter=",")
x=x.astype(int)
#x[x>=15]=255
#x[x<15]=1
x.shape

In [ ]:
from skimage.graph import MCP_Geometric
index=2
start_x=citydata['xid'][0]
start_y=citydata['yid'][0]
end_x=citydata['xid'][index]
end_y=citydata['yid'][index]
mcp_class=MCP_Geometric
m = mcp_class(x, fully_connected=False)
costs, traceback_array = m.find_costs([(start_x,start_y)], [(end_x,end_y)])
traceback_array.shape
costs.min()

In [ ]:
import matplotlib.pyplot as plt
indices, weight = route_through_array(x, (start_y,start_x), (end_y,end_x),geometric=True,fully_connected=False)
fig=plt.figure(figsize=(20,10))
cax=plt.imshow(x.T, cmap='hot', interpolation='nearest')
fig.colorbar(cax, ticks=[-1, 0, 1], orientation='horizontal')
for item in indices:
    plt.plot(item[1],item[0], 'bo')
plt.show()

In [ ]:

len(x[x>15])

In [ ]:
end_x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Maze(object):
	def __init__(self, grid_size, start=[], goal = []):
		self.grid_size = grid_size
		self.num_actions = 4  
		# four actions in each state -- up, right, bottom, left
		self.free_states = free_states
		self.start = start
		self.goal = goal
#		self.maze = np.zeros((grid_size,grid_size))
		self.maze = np.random.random_integers(0,20,(grid_size,grid_size))
		self.free_states = []
#		for i in self.free_states:
#			self.maze[i[0]][i[1]] = 1

	def reset(self):
		# reset the environment
#		self.start_index = np.random.randint(0,len(self.free_states))
		self.curr_state = self.start

	def state(self):
		return self.curr_state

	def draw(self, path = ""):
		# draw the maze configiration
		self.grid = self.maze
        #np.zeros((self.grid_size, self.grid_size))
#		for i in self.maze:
#			self.grid[i[1]][i[0]] = 0.5
		self.grid[self.goal[1]][self.goal[0]] = 1
		plt.figure(0)
		plt.clf()
		plt.imshow(self.grid, interpolation='none', cmap='gray')
		plt.savefig(path + "maze.png")

	def act(self, action):
		if(action == -1):
			self.next_state = self.curr_state
		elif(action == 0):
			self.next_state = [self.curr_state[0]-1,self.curr_state[1]]
		elif(action == 1):
			self.next_state = [self.curr_state[0]+1,self.curr_state[1]]
		elif(action == 2):
			self.next_state = [self.curr_state[0],self.curr_state[1]+1]
		elif(action == 3):
			self.next_state = [self.curr_state[0],self.curr_state[1]-1]

#		if ((self.next_state in self.free_states) or (self.next_state == self.goal)):
		self.curr_state = self.next_state
		self.reward = int(self.maze[self.curr_state[0],self.curr_state[1]] < 15)
#		else:
#			self.next_state = self.curr_state
#			self.reward = 0

#		if(self.next_state == self.goal):
#			self.reward = 1
#			self.game_over = True
#		else:
		self.game_over = self.reward == 0

		return self.next_state, self.reward, self.game_over

In [ ]:
env = Maze(15, (1,1), (14,14))
env.draw("task/")
env.reset()
env.act(2)
#env.maze

In [ ]:
Q = [[[0,0,0,0] for i in range(grid_size)] for j in range(grid_size)]
alpha = 0.6
discount = 0.9
for i in range(1000):
    action = np.random.randint(0, env.num_actions)
    curr_state = env.curr_state
    next_state, reward, game_over = env.act(action)
# Q-learning update
    x = alpha*(reward + discount*max(Q[next_state[0]][next_state[1]]) - Q[curr_state[0]][curr_state[1]][action])
    Q[curr_state[0]][curr_state[1]][action] = Q[curr_state[0]][curr_state[1]][action] + x
#    print(env.act(action))
    


In [ ]:
plot_policy(Q,1)

In [ ]:
#from maze import Maze
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

# FLAG_policy = True
FLAG_policy = False
# flag to plot polciy after each episode

def plot_policy(Q, num):
	plt.figure(1)
	plt.clf()
	grid_size = len(Q)
	plot =  [[max(Q[i][j]) for i in range(grid_size)] for j in range(grid_size)]
	plt.imshow(plot, interpolation='none', cmap='gray')
	if num == 0:
		plt.savefig("policies/final_policy.png")
	else:
		plt.savefig("policies/policy_%d.png" % (num))

def change(Q1, Q2, env):
	thres = 0.0 
	for i in env.free_states:
		prev_val = sum(Q1[i[0]][i[1]])
		new_val = sum(Q2[i[0]][i[1]])
		if(abs(prev_val - new_val) > thres):
			change = 1
			break
		else:
			change = 0
	return change

def learnTask(env, Q, epsilon = 0.3, alpha = 0.6, discount = 0.9):
	grid_size = len(Q)
	num_actions = env.num_actions

	## Learning source task

	tot_step = 0 # to count total no. of steps
	episode = 0 # to count total no. of episodes
	not_change_count = 0 # to check if Q function is changed or not
	change_no = 5 # required number of episodes for which Q function should be unchanged before stopping

	while (True):
		env.reset()
		game_over = False
		max_step = 100  # max number of steps for an episode, after max_iter steps, the episode ends
		step = 0
		episode += 1
		Q2 = deepcopy(Q)
		while not (game_over or step > max_step):
			step += 1
			curr_state = env.state()
			if np.random.rand() <= epsilon:  # epsilon-greedy policy
				action = np.random.randint(0, num_actions)
			else:
				if(max(Q[curr_state[0]][curr_state[1]]) == min(Q[curr_state[0]][curr_state[1]])):
					action = -1
					# if Q[] function is unable to select action, then no action taken
				else:
					action = np.argmax(Q[curr_state[0]][curr_state[1]])
					# best action from Q table
			next_state, reward, game_over = env.act(action)
			# Q-learning update
			Q[curr_state[0]][curr_state[1]][action] = Q[curr_state[0]][curr_state[1]][action] + alpha*(reward + discount*max(Q[next_state[0]][next_state[1]]) - Q[curr_state[0]][curr_state[1]][action])
		tot_step += step
		if (step > max_step):
			not_change_count = 0
		elif not change(Q, Q2, env):
			not_change_count += 1
			if(not_change_count == change_no):
				break
		else:
			not_change_count = 0

		if FLAG_policy == True:
			if (episode-1)%50 == 0:
				plot_policy(Q, episode)
	print("Total no. of episodes: %d" %episode)
	print("Total no. of steps: %d" %tot_step)

if __name__ == "__main__":

	print("Initializing")

	grid_size = 11
	# gridsize defines the area of the maze (gridsize X gridsize)
	# number of states in the maze are (gridsize X gridsize)

	free_states = [[ 1,7 ] , [ 0,7 ] , [ 0,6 ] , [ 0,5 ] , [ 0,4 ] , [ 0,3 ] , [ 1,3 ] , [ 2,1 ] , [ 2,2 ] , [ 2,3 ] , [ 2,4 ] , [ 3,8 ] , [ 3,7 ] , [ 3,6 ] , [ 3,5 ] , [ 3,4 ] , [ 4,4 ] , [ 0,10 ] , [ 1,10 ] , [ 2,10 ] , [ 3,10 ] , [ 4,10 ] , [ 5,10 ] , [ 5,9 ] , [ 5,8 ] , [ 5,7 ] , [ 5,6 ] , [ 5,5 ] , [ 5,4 ] , [ 6,4 ] , [ 7,4 ] , [ 8,4 ] , [ 7,10 ] , [ 8,10 ] , [ 9,10 ] , [ 10,10 ] , [ 10,9 ] , [ 10,8 ] , [ 10,7 ] , [ 10,6 ] , [ 8,5 ] , [ 9,6 ] , [ 8,6 ] , [ 8,7 ]]
	# free states define the set of states which are accessible to the agent.

	goal = [8,8]
	# goal defines the state which is the goal position of the maze task

	Q = [[[0,0,0,0] for i in range(grid_size)] for j in range(grid_size)]
	# Q table for storing value corresponing to each action-state pair

	print("Creating Maze Environment")
	env = Maze(grid_size, free_states, goal)
	# creating an instance of maze class

	print("Drawing the Maze Task")
	env.draw("task/")
	# plot the maze with the specified free_states and goal positions
	# in task folder

	print("Learning the policy")
	learnTask(env, Q)
	#learn the policy using Q-learning

	print("Plotting the learned policy")
	plot_policy(Q, 0)
	# plot the action-value function 

	print("Done! checkout task and policies folders")

In [ ]:
import os
cwd = os.getcwd()
cwd

In [ ]:
free_states = [[ 1,7 ] , [ 0,7 ] , [ 0,6 ] , [ 0,5 ] , [ 0,4 ] , [ 0,3 ] , [ 1,3 ] , [ 2,1 ] , [ 2,2 ] , [ 2,3 ] , [ 2,4 ] , [ 3,8 ] , [ 3,7 ] , [ 3,6 ] , [ 3,5 ] , [ 3,4 ] , [ 4,4 ] , [ 0,10 ] , [ 1,10 ] , [ 2,10 ] , [ 3,10 ] , [ 4,10 ] , [ 5,10 ] , [ 5,9 ] , [ 5,8 ] , [ 5,7 ] , [ 5,6 ] , [ 5,5 ] , [ 5,4 ] , [ 6,4 ] , [ 7,4 ] , [ 8,4 ] , [ 7,10 ] , [ 8,10 ] , [ 9,10 ] , [ 10,10 ] , [ 10,9 ] , [ 10,8 ] , [ 10,7 ] , [ 10,6 ] , [ 8,5 ] , [ 9,6 ] , [ 8,6 ] , [ 8,7 ]]

In [ ]:
len(free_states[0])

In [26]:
import gym
import numpy as np
import matplotlib as plt
from gym.envs.registration import registry, register, make, spec
from gym import spaces
def manhattan_distance(start, end):
    sx, sy = start
    ex, ey = end
    return abs(ex - sx) + abs(ey - sy)
class ZeppelinEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 50
    }
    def __init__(self):
        self.gravity = 9.8
        self.action_space = spaces.Discrete(5)
        width = 4
        height = 3
        start_pos = (0,0)
        end_pos = (width, height)
        self.grid = np.random.random_integers(0,25,(width,height))
        self.goal = end_pos
        self.state = start_pos
    def _render(self,mode,close):
        #self.grid[self.goal[1]][self.goal[0]] = 1
#        plt.figure(0)
        #plt.clf()
        plt.imshow(self.grid, interpolation='none', cmap='gray')
#        plt.imshow(costSurfaceArray, cmap='hot', interpolation='nearest')
        #plt.savefig(path + "maze.png")

    def _step(self, action):
        reward = 1
        state = self.state
        if action == 0:
            next_state = state
        elif action == 1:# up
            next_state = (state[0],state[1]+1)
        elif action == 2:#left
            next_state = (state[0]-1,state[1])
        elif action == 3:#right
            next_state = (state[0]+1,state[1])
        elif action == 4:# down
            next_state = (state[0],state[1]-1)
        wind_speed = self.grid[next_state[0],next_state[1]]
        if next_state[0] >= 0 and next_state[1] >= 0 and wind_speed < 15:
            self.state = next_state
        distance = manhattan_distance(state, self.goal)
        reward = distance * int(wind_speed<15)
        print("distance: " + str(distance) + ", speed: " + str(wind_speed))
        done = False
        return self.state, reward, done, {}
    def _reset(self):
        print("do nothing")


In [27]:
env = gym.make('zeppelin-v2')
observation = env.reset()
for i in range(10):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
#    print((observation, reward, done, info))
env.render()

do nothing
distance: 7, speed: 11
distance: 7, speed: 11
distance: 7, speed: 11
distance: 7, speed: 5
distance: 6, speed: 5
distance: 6, speed: 0
distance: 5, speed: 8
distance: 4, speed: 0
distance: 5, speed: 8
distance: 4, speed: 6


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. Please call randint(0, 25 + 1) instead


AttributeError: module 'matplotlib' has no attribute 'imshow'

In [28]:
import matplotlib as plt
plt.imshow

AttributeError: module 'matplotlib' has no attribute 'imshow'

In [5]:
register(
    id='zeppelin-v2',
    entry_point=__name__ +':ZeppelinEnv',
    max_episode_steps=200,
    reward_threshold=25.0,
)
